In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)


In [12]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py')
# cfg = Config.fromfile('./configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

print(cfg.data.train.pipeline)
cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)


cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/universe101'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.evaluation = dict(interval=1, metric="bbox", save_best="bbox_mAP_50")
cfg.log_config.hooks[1].init_kwargs.project = 'mask_swin'
cfg.log_config.hooks[1].init_kwargs.name = 'mask_swin_0'

print(cfg.lr_config)

[{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'Resize', 'img_scale': (512, 512), 'keep_ratio': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Albu', 'transforms': [{'type': 'OneOf', 'transforms': [{'type': 'Rotate', 'limit': 10, 'p': 0.6}, {'type': 'HorizontalFlip', 'p': 0.6}], 'p': 0.1}, {'type': 'ShiftScaleRotate', 'p': 0.5}], 'bbox_params': {'type': 'BboxParams', 'format': 'pascal_voc', 'label_fields': ['gt_labels'], 'min_visibility': 0.0, 'filter_lost_elements': True}, 'keymap': {'img': 'image', 'gt_bboxes': 'bboxes', 'gt_masks': 'masks'}, 'update_pad_shape': False, 'skip_img_without_anno': True}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]
{'policy': 'step', 'warmup': 'linear', 'warmup_iters': 5, 'warmu